# Playground

In [7]:
# Parameters
wikidata_home = "/Users/pedroszekely/Downloads/kypher"
wikidata_parts_folder = "/Users/pedroszekely/Downloads/kypher/useful_wikidata_files"
home = "/Users/pedroszekely/Downloads/kypher"
cache_folder = "/Users/pedroszekely/Downloads/kypher"
output_folder = "/Users/pedroszekely/Downloads/scratch"
delete_database = "no"

In [8]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

# from IPython.display import display, HTML, Image
# from pandas_profiling import ProfileReport

### Set up environment variables and folders that we need

In [9]:
# folder containing wikidata broken down into smaller files.
os.environ['WIKIDATA_PARTS'] = wikidata_parts_folder
# path of folder where the wikidata parts folder is stored.
os.environ['WIKIDATA_HOME'] = wikidata_home
os.environ['KYPHER'] = home
os.environ['OUT'] = output_folder
# kgtk command to run
os.environ['kgtk'] = "kgtk"
os.environ['kgtk'] = "time kgtk --debug"
# absolute path of the db
os.environ['STORE'] = "{}/Q44.wikidata.sqlite3.db".format(cache_folder)

In [10]:
cd $home

/Users/pedroszekely/Downloads/kypher


In [11]:
def run_command(cmd, substitution_dictionary = {}):
    """Run a templetized command."""
    for k, v in substitution_dictionary.items():
        cmd = cmd.replace(k, v)
    
    print(cmd)
    output = subprocess.run([cmd], shell=True, universal_newlines=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(output.stdout)
    print(output.stderr)
    #print(output.returncode)

In [12]:
# data = pd.read_csv(os.environ['OUT']+"/test.tsv", delimiter='\t')

In [1]:
def bar_chart(data, x_column, y_column):
    """Construct a simple bar chart with two properties"""
    bars = alt.Chart(data).mark_bar().encode(
        y=alt.Y(y_column, sort='-x'),
        x=x_column
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=x_column
    )

    return (bars + text)

## Remove a list of Qnodes from a KG

Let's try to remove country from the Q44 KG

In [8]:
!wd u Q6256 Q112099 Q20181813

id Q6256
Label country
Description distinct region in geography; a broad term that can include political divisions or regions associated with distinct political characteristics
subclass of (P279): political territorial entity (Q1048835)

id Q112099
Label island nation
Description state whose primary territory consists of one or more islands or parts of islands
subclass of (P279): state (Q7275) | country (Q6256)

id Q20181813
Label colonial power
Description country that controls colonies
subclass of (P279): sovereign state (Q3624078)


Find all the countries in the Q44 KG

In [9]:
!$kgtk query -i $KYPHER/Q44/Q44.part.wikibase-item.tsv.gz -i $KYPHER/Q44/Q44.P279star.tsv.gz --graph-cache $STORE \
--match 'Q44: (n1)-[:P31]->(n2), P279star: (n2)-[:P279star]->(:Q6256)' \
--return 'distinct n1 as node1, n2 as node2' \
> $OUT/Q44.items.remove.tsv

[2020-10-18 22:29:34 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "node1", graph_1_c1."node2" "node2"
     FROM graph_1 AS graph_1_c1, graph_2 AS graph_2_c2
     WHERE graph_1_c1."label"=?
     AND graph_2_c2."label"=?
     AND graph_2_c2."node2"=?
     AND graph_1_c1."node2"=graph_2_c2."node1"
  PARAS: ['P31', 'P279star', 'Q6256']
---------------------------------------------
        0.94 real         0.55 user         0.16 sys


In [10]:
!wc $OUT/Q44.items.remove.tsv

     162     324    2169 /Users/pedroszekely/Downloads/scratch/Q44.items.remove.tsv


We have 162 countries to remove, let's get all their edges that are present in Q44.

To do this we need to scan all the partitions, so we need to concatenate them first.

In [11]:
!exa -l $KYPHER/Q44/*.tsv.gz

.rw-r--r-- 416k pedroszekely 16 Oct 22:39 /Users/pedroszekely/Downloads/kypher/Q44/Q44.alias.en.tsv.gz
.rw-r--r-- 490k pedroszekely 16 Oct 22:39 /Users/pedroszekely/Downloads/kypher/Q44/Q44.description.en.tsv.gz
.rw-r--r-- 456k pedroszekely 16 Oct 22:39 /Users/pedroszekely/Downloads/kypher/Q44/Q44.label.en.tsv.gz
.rw-r--r-- 1.3M pedroszekely 16 Oct 22:39 /Users/pedroszekely/Downloads/kypher/Q44/Q44.P279star.tsv.gz
.rw-r--r--  22k pedroszekely 16 Oct 22:38 /Users/pedroszekely/Downloads/kypher/Q44/Q44.part.commonsMedia.tsv.gz
.rw-r--r--  95k pedroszekely 16 Oct 22:38 /Users/pedroszekely/Downloads/kypher/Q44/Q44.part.external-id.tsv.gz
.rw-r--r--  938 pedroszekely 16 Oct 22:38 /Users/pedroszekely/Downloads/kypher/Q44/Q44.part.geo-shape.tsv.gz
.rw-r--r-- 6.4k pedroszekely 16 Oct 22:38 /Users/pedroszekely/Downloads/kypher/Q44/Q44.part.globe-coordinate.tsv.gz
.rw-r--r--   62 pedroszekely 16 Oct 22:38 /Users/pedroszekely/Downloads/kypher/Q44/Q44.part.math.tsv.gz
.rw-r--r--  38k pedroszekely 1

In [12]:
!$kgtk cat \
-i $KYPHER/Q44/Q44.part.*.tsv.gz \
-i $KYPHER/Q44/Q44.alias.en.tsv.gz \
-i $KYPHER/Q44/Q44.description.en.tsv.gz \
-i $KYPHER/Q44/Q44.label.en.tsv.gz \
> $OUT/Q44.all.edges.tsv

        1.70 real         1.41 user         0.14 sys


In [13]:
!$kgtk query -i $OUT/Q44.items.remove.tsv -i $OUT/Q44.all.edges.tsv --graph-cache $STORE \
--match 'remove: (n1)-[]->(), all: (n1)-[l]->(n2)' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
--order-by l \
-o $OUT/Q44.edges.remove.tsv

[2020-10-18 22:29:48 sqlstore]: DROP graph data table graph_3 from /Users/pedroszekely/Downloads/scratch/Q44.items.remove.tsv
[2020-10-18 22:29:48 sqlstore]: IMPORT graph directly into table graph_12 from /Users/pedroszekely/Downloads/scratch/Q44.items.remove.tsv ...
[2020-10-18 22:29:48 sqlstore]: DROP graph data table graph_5 from /Users/pedroszekely/Downloads/scratch/Q44.all.edges.tsv
[2020-10-18 22:29:48 sqlstore]: IMPORT graph directly into table graph_13 from /Users/pedroszekely/Downloads/scratch/Q44.all.edges.tsv ...
[2020-10-18 22:29:49 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_13_c2."node1" "node1", graph_13_c2."label" "label", graph_13_c2."node2" "node2", graph_13_c2."id" "id"
     FROM graph_12 AS graph_12_c1, graph_13 AS graph_13_c2
     WHERE graph_12_c1."node1"=graph_13_c2."node1"
     ORDER BY graph_13_c2."id" ASC
  PARAS: []
---------------------------------------------
[2020-10-18 22:29:49 sqlstore]: CREATE INDEX on 

In [14]:
!head $OUT/Q44.edges.remove.tsv

node1	label	node2	id
Q1011	P1036	"2--6658"	Q1011-P1036-1
Q1011	P1081	+0.572	Q1011-P1081-1
Q1011	P1081	+0.585	Q1011-P1081-10
Q1011	P1081	+0.589	Q1011-P1081-11
Q1011	P1081	+0.592	Q1011-P1081-12
Q1011	P1081	+0.598	Q1011-P1081-13
Q1011	P1081	+0.613	Q1011-P1081-14
Q1011	P1081	+0.619	Q1011-P1081-15
Q1011	P1081	+0.625	Q1011-P1081-16


In [15]:
!$kgtk ifnotexists -i $OUT/Q44.all.edges.tsv --filter-on $OUT/Q44.edges.remove.tsv \
> $OUT/Q44.trimmed.edges.tsv

        1.58 real         1.40 user         0.13 sys


In [16]:
%%bash
for f in `ls $OUT/Q44*`
do
 wc -l $f
done

  163405 /Users/pedroszekely/Downloads/scratch/Q44.all.edges.tsv
   55382 /Users/pedroszekely/Downloads/scratch/Q44.edges.remove.tsv
     162 /Users/pedroszekely/Downloads/scratch/Q44.items.remove.tsv
    4988 /Users/pedroszekely/Downloads/scratch/Q44.items.tsv
     360 /Users/pedroszekely/Downloads/scratch/Q44.language.distribution.tsv
    2779 /Users/pedroszekely/Downloads/scratch/Q44.trimmed.clusters.tsv
  107147 /Users/pedroszekely/Downloads/scratch/Q44.trimmed.edges.tsv
    4988 /Users/pedroszekely/Downloads/scratch/Q44.trimmed.edges.wikibase-item.tsv


In [17]:
!head $OUT/Q44.trimmed.edges.tsv

id	node1	label	node2
Q1000597-P18-1	Q1000597	P18	"Town Hall - geograph.org.uk - 352398.jpg"
Q1017471-P18-1	Q1017471	P18	"Bush beer.jpg"
Q1020773-P18-1	Q1020773	P18	"Tecate 003.jpg"
Q1026242-P154-1	Q1026242	P154	"Calanda Bierdeckel.jpg"
Q10304159-P154-1	Q10304159	P154	"Itaipava-logo.gif"
Q1035257-P18-1	Q1035257	P18	"Carapils.jpg"
Q10507704-P18-1	Q10507704	P18	"Gotlandsdricka.jpg"
Q1050906-P18-1	Q1050906	P18	"Abtbier 12.jpg"
Q1056430-P18-1	Q1056430	P18	"Cerveceria.cuahotemoc.monterrey.ncs.jpg"


### Let's compute connected components on the results to see if we created islands

First we need to create a file with only wikibase-item edges and calculate the connected components on this file

In [18]:
!$kgtk query -i $OUT/Q44.trimmed.edges.tsv -i $KYPHER/Q44/Q44.part.wikibase-item.tsv.gz --graph-cache $STORE \
--match 'trimmed: (n1)-[l]->(n2), wikibase: (n1)-[]-(n2)' \
--return 'l, n1, l.label, n2'\
-o $OUT/Q44.trimmed.edges.wikibase-item.tsv

[2020-10-18 22:30:07 sqlstore]: DROP graph data table graph_7 from /Users/pedroszekely/Downloads/scratch/Q44.trimmed.edges.tsv
[2020-10-18 22:30:07 sqlstore]: IMPORT graph directly into table graph_14 from /Users/pedroszekely/Downloads/scratch/Q44.trimmed.edges.tsv ...
[2020-10-18 22:30:08 query]: SQL Translation:
---------------------------------------------
  SELECT graph_14_c1."id", graph_14_c1."node1", graph_14_c1."label", graph_14_c1."node2"
     FROM graph_1 AS graph_1_c2, graph_14 AS graph_14_c1
     WHERE graph_14_c1."node1"=graph_1_c2."node1"
     AND graph_14_c1."node2"=graph_1_c2."node2"
  PARAS: []
---------------------------------------------
[2020-10-18 22:30:08 sqlstore]: CREATE INDEX on table graph_14 column node1 ...
[2020-10-18 22:30:08 sqlstore]: ANALYZE INDEX on table graph_14 column node1 ...
[2020-10-18 22:30:08 sqlstore]: CREATE INDEX on table graph_14 column node2 ...
[2020-10-18 22:30:08 sqlstore]: ANALYZE INDEX on table graph_14 column node2 ...
        1.91 r

In [19]:
!$kgtk connected-components -i $OUT/Q44.trimmed.edges.wikibase-item.tsv \
--undirected \
--minimum-cluster-size 1 \
--cluster-name-method numbered \
-o $OUT/Q44.trimmed.clusters.tsv 

        2.20 real         0.73 user         0.19 sys


In [20]:
!wc $OUT/Q44.trimmed.clusters.tsv 

    2779    8337   84897 /Users/pedroszekely/Downloads/scratch/Q44.trimmed.clusters.tsv


In [21]:
!head $OUT/Q44.trimmed.clusters.tsv 

node1	label	node2
Q100	connected_component	0
Q1000115	connected_component	0
Q100019	connected_component	0
Q10002	connected_component	0
Q10002198	connected_component	0
Q1000597	connected_component	0
Q100188	connected_component	0
Q10023446	connected_component	0
Q10072884	connected_component	0


In [22]:
!$kgtk query -i $OUT/Q44.trimmed.clusters.tsv --graph-cache $STORE \
--match '(n1)-[]-(n2)' \
--return 'distinct n2, count(distinct n1)' \
--order-by 'count(distinct n1) desc' \
--limit 10

[2020-10-18 22:30:17 sqlstore]: DROP graph data table graph_8 from /Users/pedroszekely/Downloads/scratch/Q44.trimmed.clusters.tsv
[2020-10-18 22:30:17 sqlstore]: IMPORT graph directly into table graph_15 from /Users/pedroszekely/Downloads/scratch/Q44.trimmed.clusters.tsv ...
[2020-10-18 22:30:17 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_15_c1."node2", count(DISTINCT graph_15_c1."node1")
     FROM graph_15 AS graph_15_c1
     GROUP BY graph_15_c1."node2"
     ORDER BY count(DISTINCT graph_15_c1."node1") DESC
     LIMIT ?
  PARAS: [10]
---------------------------------------------
node2	count(DISTINCT graph_15_c1."node1")
0	2778
        0.75 real         0.58 user         0.14 sys


There is a single cluster, this is what we wanted

### Count instances of all classes

In [169]:
!$kgtk query -i $WIKIDATA_PARTS/all.P31.tsv.gz --graph-cache $STORE \
--match '(n1)-[:P31]-(n2)' \
--return 'distinct n2 as class, "P1114" as label, count(distinct n1) as count' \
--order-by 'count(distinct n1) desc' \
-o $OUT/custom.all.P31.count.tsv

[2020-10-18 17:39:23 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_9_c1."node2" "class", ? "label", count(DISTINCT graph_9_c1."node1") "count"
     FROM graph_9 AS graph_9_c1
     WHERE graph_9_c1."label"=?
     GROUP BY class, label
     ORDER BY count(DISTINCT graph_9_c1."node1") DESC
  PARAS: ['P1114', 'P31']
---------------------------------------------
      363.60 real       128.95 user        92.06 sys


In [170]:
!head $OUT/custom.all.P31.count.tsv

class	label	count
Q13442814	P1114	35933550
Q5	P1114	8064154
Q523	P1114	3297566
Q16521	P1114	2745073
Q318	P1114	2102876
Q7318358	P1114	2068766
Q7187	P1114	1196161
Q11173	P1114	1063060
Q8054	P1114	979961


Argh, the column names are not standard for an edge file. Kypher does not allow me to name the columns the way I want as `node1, label, node2`.
We need to rename the columns and add an `id` column, otherwise Kypher is unhappy

In [182]:
!$kgtk \
    rename-columns --mode NONE -i $OUT/custom.all.P31.count.tsv --output-columns node1 label node2 \
    / add-id --mode NONE --id-style node1-label-node2 \
    > $OUT/all.P31.count.tsv

        1.89 real         2.35 user         0.40 sys


In [184]:
!head $OUT/all.P31.count.tsv | column -t -s $'\t' 

node1      label  node2     id
Q13442814  P1114  35933550  Q13442814-P1114-35933550
Q5         P1114  8064154   Q5-P1114-8064154
Q523       P1114  3297566   Q523-P1114-3297566
Q16521     P1114  2745073   Q16521-P1114-2745073
Q318       P1114  2102876   Q318-P1114-2102876
Q7318358   P1114  2068766   Q7318358-P1114-2068766
Q7187      P1114  1196161   Q7187-P1114-1196161
Q11173     P1114  1063060   Q11173-P1114-1063060
Q8054      P1114  979961    Q8054-P1114-979961


Add labels to the file so we can see the names of the classes

In [189]:
!$kgtk query -i $OUT/all.P31.count.tsv -i $WIKIDATA_PARTS/part.label.en.tsv.gz --graph-cache $STORE \
--match 'P31: (n1)-[l]-(n2), label: (n1)-[:label]->(label)' \
--where 'label.kgtk_lqstring_lang_suffix = "en"' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id, label as `node1;label`' \
--order-by 'cast(n2, int) desc' \
-o $OUT/all.P31.count.labeled.tsv

[2020-10-18 17:59:55 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_11_c2."node1" "node1", graph_10_c1."label" "label", graph_10_c1."node2" "node2", graph_10_c1."id" "id", graph_11_c2."node2" "node1;label"
     FROM graph_10 AS graph_10_c1, graph_11 AS graph_11_c2
     WHERE graph_11_c2."label"=?
     AND graph_10_c1."node1"=graph_11_c2."node1"
     AND (kgtk_lqstring_lang_suffix(graph_11_c2."node2") = ?)
     ORDER BY CAST(graph_10_c1."node2" AS int) DESC
  PARAS: ['label', 'en']
---------------------------------------------
        2.13 real         1.53 user         0.57 sys


In [192]:
!head -50 $OUT/all.P31.count.labeled.tsv | column -t -s $'\t' 

node1      label  node2     id                        node1;label
Q13442814  P1114  35933550  Q13442814-P1114-35933550  'scholarly article'@en
Q5         P1114  8064154   Q5-P1114-8064154          'human'@en
Q523       P1114  3297566   Q523-P1114-3297566        'star'@en
Q16521     P1114  2745073   Q16521-P1114-2745073      'taxon'@en
Q318       P1114  2102876   Q318-P1114-2102876        'galaxy'@en
Q7318358   P1114  2068766   Q7318358-P1114-2068766    'review article'@en
Q7187      P1114  1196161   Q7187-P1114-1196161       'gene'@en
Q11173     P1114  1063060   Q11173-P1114-1063060      'chemical compound'@en
Q8054      P1114  979961    Q8054-P1114-979961        'protein'@en
Q486972    P1114  609721    Q486972-P1114-609721      'human settlement'@en
Q13100073  P1114  588509    Q13100073-P1114-588509    'village-level division in China'@en
Q8502      P1114  525197    Q8502-P1114-525197        'mountain'@en
Q871232    P1114  512908    Q871232-P1114-512908      'editorial'@en
Q3305213   

In [10]:
!head -20 $OUT/all.P31.count.labeled.tsv > $OUT/temp.top-classes.50.tsv

In [20]:
data = pd.read_csv(os.environ['OUT']+"/temp.top-classes.50.tsv", delimiter='\t')
bar_chart('node2', 'node1;label')

alt.LayerChart(...)

In [21]:
!gzcat $WIKIDATA_PARTS/almost.all.edges.tsv.gz | wc

 1443196924 7311117754 80501996248


In [195]:
!wdtaxonomy -r Q523 Q318

star (Q523) •227 ×3296581
├──astronomical object (Q6999) •88 ×25626 ↑
│  ├──physical object (Q223557) •46 ×105
│  │  └──concrete object (Q4406616) •11 ×313
│  │     └──object (Q488383) •39 ×697 ↑
│  │        └──entity (Q35120) •46 ×31 ↑
│  ├──space object (Q4235019) •3
│  │  ╘══object (Q488383) •39 ×697 ↑ …
│  └──location (Q17334923) ×196
│     └──geographic entity (Q27096213) ×53
│        └──spatial entity (Q58416391)
│           └──spatio-temporal entity (Q58415929)
│              ╘══entity (Q35120) •46 ×31 ↑ …
└──fusor (Q1027098) •12
   ╘══astronomical object (Q6999) •88 ×25626 ↑ …


In [196]:
!wdtaxonomy -r Q318

galaxy (Q318) •164 ×2101227
└──deep-sky object (Q249389) •27 ×1
   └──astronomical object (Q6999) •88 ×25626
      ├──physical object (Q223557) •46 ×105
      │  └──concrete object (Q4406616) •11 ×313
      │     └──object (Q488383) •39 ×697 ↑
      │        └──entity (Q35120) •46 ×31 ↑
      ├──space object (Q4235019) •3
      │  ╘══object (Q488383) •39 ×697 ↑ …
      └──location (Q17334923) ×196
         └──geographic entity (Q27096213) ×53
            └──spatial entity (Q58416391)
               └──spatio-temporal entity (Q58415929)
                  ╘══entity (Q35120) •46 ×31 ↑ …


In [201]:
!wdtaxonomy -r Q3504248

inner planet (Q3504248) •16 ×3
├──terrestrial planet (Q128207) •78 ×12
│  └──planet (Q634) •226 ↑
│     ├──planemo (Q400144) •20
│     │  ├──substellar object (Q3132741) •15 ×3 ↑
│     │  │  └──astronomical object (Q6999) •88 ×25626 ↑↑
│     │  │     ├──physical object (Q223557) •46 ×105
│     │  │     │  └──concrete object (Q4406616) •11 ×313
│     │  │     │     └──object (Q488383) •39 ×697 ↑
│     │  │     │        └──entity (Q35120) •46 ×31 ↑
│     │  │     ├──space object (Q4235019) •3
│     │  │     │  ╘══object (Q488383) •39 ×697 ↑ …
│     │  │     └──location (Q17334923) ×196
│     │  │        └──geographic entity (Q27096213) ×53
│     │  │           └──spatial entity (Q58416391)
│     │  │              └──spatio-temporal entity (Q58415929)
│     │  │                 ╘══entity (Q35120) •46 ×31 ↑ …
│     │  └──planetary body (Q16873378) •3 ↑
│     │     ╞══substellar object (Q3132741) •15 ×3 ↑ …
│     │     └──secondary body (Q15731960) •3
│     │        ╘══astronomical object (

In [134]:
!exa -l $WIKIDATA_PARTS/

.rw-------  333M pedroszekely 28 Sep 15:16 all.isa.tsv.gz
.rw-------  522M pedroszekely 26 Sep 21:54 all.P31.tsv.gz
.rw-------  542M pedroszekely 26 Sep 22:16 all.P31_P279.tsv.gz
.rw-------   20M pedroszekely 26 Sep 21:57 all.P279.tsv.gz
.rw-------  541M pedroszekely 28 Sep 12:18 all.P279star.tsv.gz
.rw-------  111M pedroszekely 25 Sep 16:22 part.alias.en.tsv.gz
.rw-------  772M pedroszekely 25 Sep  9:10 part.alias.tsv.gz
.rw-------  109M pedroszekely 26 Sep  0:31 part.commonsMedia.tsv.gz
.rw-------  1.0G pedroszekely 25 Sep 17:19 part.description.en.tsv.gz
.rw-------   21G pedroszekely 25 Sep 13:58 part.description.tsv.gz
.rw-------  2.0G pedroszekely 26 Sep  0:21 part.external-id.tsv.gz
.rw-------  424k pedroszekely 26 Sep  1:19 part.geo-shape.tsv.gz
.rw-------  127M pedroszekely 26 Sep  0:42 part.globe-coordinate.tsv.gz
.rw-------  2.4G pedroszekely 25 Sep 16:11 part.label.en.tsv.gz
.rw-------  6.0G pedroszekely 25 Sep  1:08 part.label.tsv.gz
.rw-------  417k pedroszekely 25 Sep 22:

In [142]:
!wd f -t p count

P1114      quantity number of instances of this subject
P17        country sovereign state of this item (not to be used for human beings)
P27        country of citizenship the object is a country that recognizes the subject as its citizen
P1001      applies to jurisdiction the item (an institution, law, public office ...) or statement belongs to or has power over or applies to the value (a territorial jurisdiction: a country, state, municipality, ...)
P495       country of origin country of origin of this item (creative work, food, phrase, product, etc.)
P1532      country for sport country a person or a team represents when playing a sport
P36        capital primary city of a country, province, state or other type of administrative territorial entity
P1376      capital of country, state, department, canton or other administrative division of which the municipality is the governmental seat
P2196      students count number of students of any type in an educational organization
P8047    

In [138]:
!wd search -h


  Usage: wd-search [options] <search>

  search entities

  Options:

    -p, --properties <properties>  request additional properties (separated by a comma) (implies verbose mode)
    -t, --type <type>              customize type: i|item, p|property, l|lexeme, f|form, s|sense (Default: item)
    -n, --limit <num>              set a custom limit (defaults to 10)
    -l, --lang <lang>              specify the language to use
    -v, --verbose                  make the output more verbose
    -j, --json                     output command results formatted as JSON
    -h, --help                     output usage information

  Examples:

    # displays a list of entities matching "Ligo"
    wd search Ligo

    # displays up to 25 results matching "Harry Potter"
    wd search Harry Potter --limit 25

    # display rich results (aka summaries)
    wd search Harry Potter --verbose

    # request additional properties (separated by a comma) to be added to the results summaries
    wd search H

## Analysis of XPO consensus

In [23]:
!exa -l $WIKIDATA_PARTS

.rw-------  333M pedroszekely 28 Sep 15:16 all.isa.tsv.gz
.rw-------  522M pedroszekely 26 Sep 21:54 all.P31.tsv.gz
.rw-------  542M pedroszekely 26 Sep 22:16 all.P31_P279.tsv.gz
.rw-------   20M pedroszekely 26 Sep 21:57 all.P279.tsv.gz
.rw-------  541M pedroszekely 28 Sep 12:18 all.P279star.tsv.gz
.rw-r--r--   10G pedroszekely 18 Oct 22:32 almost.all.edges.tsv.gz
.rw-------  111M pedroszekely 25 Sep 16:22 part.alias.en.tsv.gz
.rw-------  772M pedroszekely 25 Sep  9:10 part.alias.tsv.gz
.rw-------  109M pedroszekely 26 Sep  0:31 part.commonsMedia.tsv.gz
.rw-------  1.0G pedroszekely 25 Sep 17:19 part.description.en.tsv.gz
.rw-------  2.0G pedroszekely 26 Sep  0:21 part.external-id.tsv.gz
.rw-------  424k pedroszekely 26 Sep  1:19 part.geo-shape.tsv.gz
.rw-------  127M pedroszekely 26 Sep  0:42 part.globe-coordinate.tsv.gz
.rw-------  2.4G pedroszekely 25 Sep 16:11 part.label.en.tsv.gz
.rw-------  6.0G pedroszekely 25 Sep  1:08 part.label.tsv.gz
.rw-------  417k pedroszekely 25 Sep 22:

### Weapons (Q728)

In [27]:
!$kgtk query -i $WIKIDATA_PARTS/all.P279star.tsv.gz -i $WIKIDATA_PARTS/part.label.en.tsv.gz --graph-cache $STORE \
--match 'P279star: (n1)-[l]-(n2:Q728), label: (n1)-[:label]->(label)' \
--where 'label.kgtk_lqstring_lang_suffix = "en"' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id, label as `node1;label`' \
--order-by 'n1' \
-o $OUT/WEA.all.tsv

[2020-10-18 22:50:52 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_11_c2."node1" "node1", graph_16_c1."label" "label", graph_16_c1."node2" "node2", graph_16_c1."id" "id", graph_11_c2."node2" "node1;label"
     FROM graph_11 AS graph_11_c2, graph_16 AS graph_16_c1
     WHERE graph_11_c2."label"=?
     AND graph_16_c1."node2"=?
     AND graph_11_c2."node1"=graph_16_c1."node1"
     AND (kgtk_lqstring_lang_suffix(graph_11_c2."node2") = ?)
     ORDER BY graph_11_c2."node1" ASC
  PARAS: ['label', 'Q728', 'en']
---------------------------------------------
        1.21 real         0.68 user         0.23 sys


In [28]:
!wc $OUT/WEA.all.tsv

    3703   23052  259900 /Users/pedroszekely/Downloads/scratch/WEA.all.tsv


In [30]:
!head $OUT/WEA.all.tsv | column -t -s $'\t' 

node1     label     node2  id                           node1;label
Q1002792  P279star  Q728   Q1002792-P279star-Q728-0000  'Type I Rifle'@en
Q1003167  P279star  Q728   Q1003167-P279star-Q728-0000  'Bujo'@en
Q1004769  P279star  Q728   Q1004769-P279star-Q728-0000  'Bumbar'@en
Q1006579  P279star  Q728   Q1006579-P279star-Q728-0000  'Robinson Armament M96 Expeditionary'@en
Q1007100  P279star  Q728   Q1007100-P279star-Q728-0000  'Gryazev-Shipunov GSh-23'@en
Q1007182  P279star  Q728   Q1007182-P279star-Q728-0000  'Gabriel'@en
Q1009623  P279star  Q728   Q1009623-P279star-Q728-0000  'bunker buster'@en
Q1010024  P279star  Q728   Q1010024-P279star-Q728-0000  'Geschützwagen'@en
Q1014037  P279star  Q728   Q1014037-P279star-Q728-0000  'Type 97'@en


In [35]:
!grep Q55942755 $OUT/WEA.all.tsv 

Q55942755	P279star	Q728	Q55942755-P279star-Q728-0000	'poison gas'@en


In [34]:
!wd u Q22731

id Q22731
Label stone
Description rock or artificial rock-like material
subclass of (P279): base material (Q214609)


In [22]:
!kgtk ifnotexists --help

usage: kgtk ifnotexists [-h] [-i INPUT_FILE] [--filter-on FILTER_FILE]
                        [-o OUTPUT_FILE] [--reject-file REJECT_FILE]
                        [--input-keys [INPUT_KEYS [INPUT_KEYS ...]]]
                        [--filter-keys [FILTER_KEYS [FILTER_KEYS ...]]]
                        [--cache-input [True|False]]
                        [--preserve-order [True|False]] [-v]

Filter a KGTK file based on whether one or more records do not exist in a second KGTK file with matching values for one or more fields.

Additional options are shown in expert help.
kgtk --expert ifnotexists --help

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT_FILE, --input-file INPUT_FILE
                        The KGTK input file. (May be omitted or '-' for
                        stdin.)
  --filter-on FILTER_FILE
                        The KGTK file to filter against. (May be omitted or
                        '-' for stdin.)
  -o OUTPUT_FILE, --outpu

Quick analysis of pharmaceutical products

In [3]:
!exa /Users/pedroszekely/Downloads/kypher/Q28885102

Q28885102.alias.en.tsv.gz               Q28885102.part.musical-notation.tsv.gz
Q28885102.description.en.tsv.gz         Q28885102.part.quantity.tsv.gz
Q28885102.everything.statistics.txt     Q28885102.part.string.tsv.gz
Q28885102.label.en.tsv.gz               Q28885102.part.time.tsv.gz
Q28885102.P279star.tsv.gz               Q28885102.part.url.tsv.gz
Q28885102.part.commonsMedia.tsv.gz      Q28885102.part.wikibase-form.tsv.gz
Q28885102.part.external-id.tsv.gz       Q28885102.part.wikibase-item.tsv.gz
Q28885102.part.geo-shape.tsv.gz         Q28885102.part.wikibase-property.tsv.gz
Q28885102.part.globe-coordinate.tsv.gz  Q28885102.properties.tsv.gz
Q28885102.part.math.tsv.gz              Q28885102.statistics.tsv.gz
Q28885102.part.monolingualtext.tsv.gz   


In [15]:
!kgtk query -i /Users/pedroszekely/Downloads/kypher/Q28885102/Q28885102.part.wikibase-item.tsv.gz  \
--match '(n1:Q18216)-[l]-(n2)' 

id	node1	label	node2


In [17]:
!wd u Q18216

id Q18216
Label aspirin
Description medication used to treat pain and decrease the risk of heart disease
instance of (P31):  chemical compound (Q11173) | medication (Q12140) | combustible powder (Q21073023) | essential medicine (Q35456) | Female reproductive toxicant (Q55427776) | developmental toxicant (Q72941151)
subclass of (P279): non-steroidal anti-inflammatory drug (Q188724)


In [20]:
!gzcat /Users/pedroszekely/Downloads/kypher/Q28885102/Q28885102.*.tsv.gz | grep Q18216

Q18216-P279star-Q1150070-0000	Q18216	P279star	Q1150070
Q18216-P279star-Q1190554-0000	Q18216	P279star	Q1190554
Q18216-P279star-Q12140-0000	Q18216	P279star	Q12140
Q18216-P279star-Q16887380-0000	Q18216	P279star	Q16887380
Q18216-P279star-Q1724915-0000	Q18216	P279star	Q1724915
Q18216-P279star-Q173235-0000	Q18216	P279star	Q173235
Q18216-P279star-Q1747785-0000	Q18216	P279star	Q1747785
Q18216-P279star-Q18216-0000	Q18216	P279star	Q18216
Q18216-P279star-Q188724-0000	Q18216	P279star	Q188724
Q18216-P279star-Q1914636-0000	Q18216	P279star	Q1914636
Q18216-P279star-Q20937557-0000	Q18216	P279star	Q20937557
Q18216-P279star-Q26907166-0000	Q18216	P279star	Q26907166
Q18216-P279star-Q28732711-0000	Q18216	P279star	Q28732711
Q18216-P279star-Q3249551-0000	Q18216	P279star	Q3249551
Q18216-P279star-Q35120-0000	Q18216	P279star	Q35120
Q18216-P279star-Q3769299-0000	Q18216	P279star	Q3769299
Q18216-P279star-Q4026292-0000	Q18216	P279star	Q4026292
Q18216-P279star-Q409205-0000	Q18216	P279star	Q409205
Q18216-P279star-Q434